# 데이터 분할, 불균형 해소

머신러닝을 통해 분석 모델을 적용하기 전 수집된 데이터가 분석 모델을 적용하는데 적합하도록 분할하고 데이터의 비율을 균일하게 조정하는 고급 전처리 기법

## #01. 준비과정

### [1] 주요 개념 정리

#### 1) 독립변수($x$)

- 결과를 예측하는데 필요한 조건값
- 일반적인 $y = ax + b$ 방정식에서 $x$에 해당함
- `○○이 ■■에 주는 영향`에서 `○○`에 해당
- `설명변수`라고 부르기도 함

#### 2) 종속변수($y$)

- 머신러닝이 예측하고자 하는 결과값
- 일반적인 $y = ax + b$ 방정식에서 $y$에 해당함
- `○○이 ■■에 주는 영향`에서 `■■`에 해당
- `반응변수`라고 부르기도 함

> 추후 뒤에서 좀 더 자세하게 설명합니다.

### [2] 패키지 참조

`imblearn` 패키지의 설치가 필요하다.


In [1]:
from pandas import read_excel
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

### [3] 샘플 데이터 가져오기

In [2]:
origin = read_excel('https://data.hossam.kr/pydata/gradeuate.xlsx')
origin

,합격여부,필기점수,학부성적,병원경력
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4
...,...,...,...,...
395,0,620,4.00,2
396,0,560,3.04,3
397,0,460,2.63,2
398,0,700,3.65,2


## #02. 데이터 분할

### [1] 독립변수, 종속변수 분리하기

In [3]:
x = origin.drop(['합격여부'], axis=1)
y = origin['합격여부']
x.shape, y.shape

((400, 3), (400,))

### [2] 훈련데이터와 검증데이터 분할하기

| 구분 | 설명 |
|---|---|
| 훈련데이터 | 머신러닝이 학습에 활용하기 위한 데이터 |
| 검증데이터 | 머신러닝의 학습 결과가 올바른지 검증하는데 사용되는 데이터 |

일반적으로 `7(훈련):3(검증)`으로 분할한다.

```python
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=int, random_state=int)
```

| 파라미터 | 설명 |
|---|---|
| x | 독립변수 |
| y | 종속변수 |
| test_size | 검증데이터의 비율 (`0~1`) |
| random_state | 랜덤시드 값 |

#### 데이터 분할하기

In [4]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=777)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((280, 3), (120, 3), (280,), (120,))

#### 분할된 훈련데이터의 종속변수 비율 확인

In [5]:
y_train.value_counts()

합격여부
0    189
1     91
Name: count, dtype: int64

## #03. 데이터 불균형 해소

데이터를 훈련데이터와 검증데이터로 나눈 후 훈련 데이터에 대해서만 수행해야 한다.

### [1] Under Sampling 방식 - Random Under Sampler

많은 비율을 차지하는 다수 집단에서 일부만 샘플링하는 방식

소수 집단의 데이터가 어느 정도 확보되었다고 여겨질 때, 다수 집단의 데이터를 줄여서 균형을 맞춘다.

다수 집단의 유의미한 데이터를 손실할 수 있다는 단점이 있다.

#### `sampling_strategy` 파라미터

| 값 | 설명 |
|--|--|
| `majority` | 다수 클래스만 다시 샘플링 |
| `not majority` | `다수 아님` - 다수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `not minority` | `소수 아님`- 소수 클래스를 제외한 모든 클래스를 다시 샘플링 |
| `all` | 모든 클래스를 다시 샘플링 |
| `auto` | 자동 처리 |

RandomUnderSampler 클래스의 주요 파라미터에 대한 설명은 다음과 같습니다:

sampling_strategy: 언더샘플링을 어떻게 수행할지 결정하는 파라미터입니다.

"majority": 다수 클래스의 샘플을 제거하여 클래스 간의 균형을 맞춥니다.
float: 특정 클래스의 비율을 지정합니다. 예를 들어, 0.5로 설정하면 샘플을 반으로 줄입니다.
random_state: 난수 생성 시 사용되는 시드(seed) 값입니다. 이 값을 설정하면 언더샘플링이나 다른 무작위 기능을 실행할 때 마다 동일한 결과를 얻을 수 있습니다.

replacement: 기본값은 False입니다. False로 설정하면 복원 추출이 아닌 비복원 추출이 이루어집니다. True로 설정하면 복원 추출이 이루어져 샘플이 여러 번 선택될 수 있습니다.

n_neighbors: 특정 클래스의 샘플을 제거할 때, 해당 클래스에서 제거할 샘플의 수를 지정합니다. 이 값이 작을수록 적은 수의 샘플이 제거됩니다.

예시 코드에서 RandomUnderSampler를 생성하고 fit_resample 메서드를 사용하여 언더샘플링을 수행할 때, sampling_strategy를 "majority"로 설정하여 다수 클래스의 샘플을 제거합니다. 다른 파라미터는 기본값이 사용되었습니다.

In [6]:
undersampler = RandomUnderSampler(sampling_strategy="majority", random_state=777)
x_under, y_under = undersampler.fit_resample(x_train, y_train)
print(x_under.shape)

y_under.value_counts()

(182, 3)


합격여부
0    91
1    91
Name: count, dtype: int64